In [1]:
# 4.4.2 Generación de demanda sintética por categoría + split-delivery

import datetime, math, os
import numpy as np
import pandas as pd

# ------------------------------------------------
# 1) Horizonte temporal (modificable fácilmente)
# ------------------------------------------------
# Número de días a simular (p. ej. 7 para una semana, 30 para un mes)
horizon_days = 7
start_date   = datetime.date.today()
end_date     = start_date + datetime.timedelta(days=horizon_days - 1)
# Lista de fechas del horizonte
dates = pd.date_range(start_date, end_date, freq='D').date

# ------------------------------------------------
# 2) Parámetros de la simulación
# ------------------------------------------------
M_base      = 100                   # pedidos base por día
size_min, size_max = 1, 5           # kg mínimo/máximo por pedido
cat_labels  = ['carne','pescado','med']
raw_targets = np.array([177.8, 96.4, 22.2])  # kg/día deseados por categoría
cat_probs   = raw_targets / raw_targets.sum()
# Factor de variación según día de la semana (0=Lun, …, 6=Dom)
factors     = {0:0.7,1:1.0,2:1.0,3:1.0,4:1.0,5:1.3,6:1.3}

# ------------------------------------------------
# 3) Carga de paradas y probabilidades de origen
# ------------------------------------------------
stops_df = pd.read_csv('../data/PARADAS_CON_HABITANTES.csv').iloc[1:]
stops    = stops_df['Name'].tolist()
w        = stops_df['HABITANTES'].values
weights  = w / w.sum()

# ------------------------------------------------
# 4) Simulación diaria y pivot a categorías
# ------------------------------------------------
records = []
for d in dates:
    # Número de pedidos ese día
    M = int(M_base * factors[d.weekday()])
    # Orígenes, tamaños y categorías de cada pedido
    origins = np.random.choice(stops, size=M, p=weights)
    sizes   = np.random.randint(size_min, size_max+1, size=M)
    cats    = np.random.choice(cat_labels, size=M, p=cat_probs)

    df = pd.DataFrame({
        'date': d,
        'stop': origins,
        'qty':  sizes,
        'category': cats
    })

    # Pivot para sumar por (date, stop, category)
    piv = (df
           .pivot_table(index=['date','stop'],
                        columns='category',
                        values='qty',
                        aggfunc='sum',
                        fill_value=0)
           .reset_index())
    # Asegurar que aparecen todas las categorías
    for c in cat_labels:
        if c not in piv:
            piv[c] = 0
    # Demanda total por parada
    piv['simulated_demand_kg'] = piv[cat_labels].sum(axis=1)

    records.append(piv[['date','stop'] + cat_labels + ['simulated_demand_kg']])

# Concatenar y guardar breakdown por categoría
sim_dem = pd.concat(records, ignore_index=True)
os.makedirs('../data', exist_ok=True)
sim_dem.to_csv('../data/simulated_demands.csv', index=False)
print("✅ Demanda por categoría guardada en ../data/simulated_demands.csv")

# ------------------------------------------------
# 5) Split-delivery: fragmentar demandas > C
# ------------------------------------------------
C = 30  # capacidad máxima por vuelo
split_rows = []
for _, row in sim_dem.iterrows():
    total = row['simulated_demand_kg']
    parts = math.ceil(total / C)
    for k in range(parts):
        qty = C if k < parts - 1 else total - C * (parts - 1)
        split_rows.append({
            'date': row['date'],
            'stop': row['stop'],
            'simulated_demand_kg': int(qty)
        })

dem_split = pd.DataFrame(split_rows)
dem_split.to_csv('../data/simulated_demands_split.csv', index=False)
print(f"✅ Split-delivery completado. Tamaño máximo en nodo = {dem_split['simulated_demand_kg'].max()} kg")


✅ Demanda por categoría guardada en ../data/simulated_demands.csv
✅ Split-delivery completado. Tamaño máximo en nodo = 30 kg
